In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import constants as const
from scipy import integrate

In [2]:
def BB_300K(wav):
    """Function calculates blackbody spectrum at 300K.

    Args:
        wav (array): Wavelength values to use to calculate blackbody spectrum in m.

    Returns:
        array: Blackbody spectrum in W/m^2/m
    """
    # Define constants
    h = const.h
    c = const.c
    k = const.k
    T = 300

    # Calculate blackbody spectrum
    a = 8 * np.pi * h * c**2
    b = h * c / (wav * k * T)
    intensity = a / ((wav**5) * (np.exp(b) - 1))

    return intensity


def EQE(E_G, wav):
    """Calculates EQE step function for a given band gap energy.

    Args:
        E_G (array): Band gap energy in Joules
        wav (array): Wavelength values to use to calculate solar spectrum in m. This should have the same length as EQE and spectrum.

    Returns:
        2D array: EQE step function.
    """
    # Define constants
    h = const.h
    c = const.c

    # Initialize EQE array
    EQE = np.zeros(shape=(len(E_G), len(wav)))

    # Calculate a step function for each band gap energy
    hc_wav = h * c / wav

    # Calculate EQE using vectorized operations
    E_G = E_G.reshape(-1, 1)
    hc_wav = hc_wav.reshape(1, -1)
    EQE = np.where(hc_wav > E_G, 1, 0)

    # Slower method that accomplishes the same thing:
    # Iterate, for each band gap energy, through the wavelengths
    # for i in range(len(E_G)):
    #     for x in range(len(wav)):
    #         if h * c / wav[x] > E_G[i]:
    #             EQE[i, x] = 1
    #         else:
    #             EQE[i, x] = 0

    return EQE


def J_G(spectrum, wav, E_G, EQE):
    """Function calculates generation current density.

    Args:
        spectrum (array): Solar spectrum data (y-axis values) in W/m^2/nm.
        wav (array): Wavelength values to use to calculate solar spectrum in m. This should have the same length as EQE and spectrum.
        E_G (array): Band gap energy in Joules
        EQE (2D array): EQE step function.

    Returns:
        array: Generation current density in A/m^2.
    """
    # Define constants
    q = const.e
    h = const.h
    c = const.c

    wav_m = wav

    photon_flux = spectrum * wav_m / (h * c)  # units of photons/m^2/s/nm
    photon_flux *= 1 / (1e-9)  # convert from nm^-1 to m^-1

    # Integrate to get 1 J_G value for each band gap energy
    J_G = np.zeros(len(E_G))
    for i in range(len(E_G)):
        J_G[i] = np.trapz(q * photon_flux * EQE[i, :], wav)
    return J_G


def J_o(EQE, BB, wav, E_G):
    """Calculates dark current density.

    Args:
        EQE (2D array): EQE step function.
        BB (array): Blackbody spectrum in W/m^2/m.
        wav (array): Wavelength values to use to calculate solar spectrum in m. This should have the same length as EQE and spectrum.
        E_G (array): Band gap energy in Joules

    Returns:
        array: Dark current density in A/m^2.
    """
    # Define constants
    q = const.e

    # Integrate to get 1 J_o value for each band gap energy
    J_o = np.zeros(len(wav))
    # for i in range(len(E_G)):
    #     J_o[i] = q * integrate.simpson(EQE[i, :] * BB, wav, dx=wav[1] - wav[0])
    for i in range(len(E_G)):
        J_o[i] = (
            2
            * 2
            * const.pi
            * q
            * const.c
            * np.trapz(
                EQE[i, :]
                * 1
                / wav**4
                * 1
                / (np.exp(const.h * const.c / (wav * const.k * 300)) - 1),
                wav,
                # dx=wav[1] - wav[0],
            )
        )

    return J_o


def V_oc(J_G, J_o):
    """Function calculates open-circuit voltage.

    Args:
        T (float): Temperature in Kelvin.
        J_G (array): Generation current density in A/m^2.
        J_o (array): Dark current density in A/m^2.

    Returns:
        array: Open-circuit voltage in V.
    """
    # Define constants
    k = const.k
    q = const.e

    # Calculate V_oc
    return (k * 300 / q) * np.log(J_G / J_o + 1)

In [3]:
# Load in spectral data
AM10 = np.loadtxt(
    "./Solar Spectra by Airmass/AM1.0.txt",
    skiprows=1,
)
AM125 = np.loadtxt(
    "./Solar Spectra by Airmass/AM1.25.txt",
    skiprows=1,
)
AM15 = np.loadtxt(
    "./Solar Spectra by Airmass/AM1.5.txt",
    skiprows=1,
)
AM175 = np.loadtxt(
    "./Solar Spectra by Airmass/AM1.75.txt",
    skiprows=1,
)
AM20 = np.loadtxt(
    "./Solar Spectra by Airmass/AM2.0.txt",
    skiprows=1,
)

In [4]:
# Define constants
E_G = np.linspace(1, 5, len(AM10[:, 0])) * const.e  # band gap energies in joules
T = 300
wav = AM10[:, 0] * 1e-9  # wavelength values in m
BB = BB_300K(
    wav
)  # blackbody spectrum in W/m^2/m using wavelength values from AM1.0 txt file

# Calculate V_oc for each band gap energy
spectrum10 = AM10[:, 3]  # solar spectrum in W/m^2/nm
V_oc_10 = V_oc(
    J_G(
        spectrum10, wav, E_G, EQE(E_G, wav)
    ),  # J_G function deals with the fact that the solar spectrum is in W/m^2/nm. For everything else, we use SI units.
    J_o(EQE(E_G, wav), BB, wav, E_G),
)

spectrum125 = AM125[:, 3]  # solar spectrum in W/m^2/nm
V_oc_125 = V_oc(
    J_G(spectrum125, wav, E_G, EQE(E_G, wav)),
    J_o(EQE(E_G, wav), BB, wav, E_G),
)

spectrum15 = AM15[:, 3]  # solar spectrum in W/m^2/nm
V_oc_15 = V_oc(
    J_G(spectrum15, wav, E_G, EQE(E_G, wav)),
    J_o(EQE(E_G, wav), BB, wav, E_G),
)

spectrum175 = AM175[:, 3]  # solar spectrum in W/m^2/nm
V_oc_175 = V_oc(
    J_G(spectrum175, wav, E_G, EQE(E_G, wav)),
    J_o(EQE(E_G, wav), BB, wav, E_G),
)

spectrum20 = AM20[:, 3]  # solar spectrum in W/m^2/nm
V_oc_20 = V_oc(
    J_G(spectrum20, wav, E_G, EQE(E_G, wav)),
    J_o(EQE(E_G, wav), BB, wav, E_G),
)

/tmp/ipykernel_5261/4276327975.py:146: RuntimeWarning: invalid value encountered in divide
  return (k * 300 / q) * np.log(J_G / J_o + 1)


In [5]:
# PGF Plots
plt.rcParams.update({"font.size": 20})

matplotlib.use("pgf")
matplotlib.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "font.serif": ["Times New Roman", "CMU Serif"],  #  Fallback to CMU Serif
        "text.usetex": True,
        "pgf.rcfonts": False,
    }
)
plt.style.use("seaborn-v0_8-bright")

In [6]:
colors = [
    "#c44601",
    "#f57600",
    "#8babf1",
    "#0073e6",
    "#5ba300",
]  # Store 5 colour-blind friendly colours

In [ ]:
# Plot J_G vs. band gap energy
plt.figure()
plt.plot(
    E_G / const.e,
    J_G(spectrum10, wav, E_G, EQE(E_G, wav)) / 10,
    label="AM0",
    color=colors[0],
)
plt.plot(
    E_G / const.e,
    J_G(spectrum125, wav, E_G, EQE(E_G, wav)) / 10,
    label="AM1.25",
    color=colors[1],
)
plt.plot(
    E_G / const.e,
    J_G(spectrum15, wav, E_G, EQE(E_G, wav)) / 10,
    label="AM1.5",
    color=colors[2],
)
plt.plot(
    E_G / const.e,
    J_G(spectrum175, wav, E_G, EQE(E_G, wav)) / 10,
    label="AM1.75",
    color=colors[3],
)
plt.plot(
    E_G / const.e,
    J_G(spectrum20, wav, E_G, EQE(E_G, wav)) / 10,
    label="AM2.0",
    color=colors[4],
)
plt.xlabel("Band Gap Energy (eV)")
plt.ylabel("$J_{sc}$ ($mA/cm^2$)")
plt.legend()
plt.savefig("J_sc.pgf", bbox_inches="tight")

# Plot J_o vs. band gap energy
plt.figure()
plt.plot(E_G / const.e, J_o(EQE(E_G, wav), BB, wav, E_G))
plt.xlabel("Band Gap Energy (eV)")
plt.ylabel("$J_{o}$ (mAcm$^{-2}$)")
plt.legend()
plt.savefig("J_o.pgf", bbox_inches="tight")

# Plot EQE for each a selection of band gap energies
plt.figure()
indexes = [0, len(E_G) // 4, len(E_G) // 2, len(E_G) - 1]
for index in indexes:
    plt.plot(
        wav * 1e9, EQE(E_G, wav)[index, :], label=f"{round(E_G[index] / const.e)} eV"
    )
plt.xlabel("Wavelength (nm)")
plt.ylabel("EQE")
plt.legend()
plt.savefig("EQE.pgf", bbox_inches="tight")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
